<a href="https://colab.research.google.com/github/bhavya26082004/python/blob/main/AD4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.2 MB/s eta 0:00:00


In [12]:
text1= "What are you saying 😂, I am the boss 😎, and why are you so 😔"

In [13]:
import re
from emot.emo_unicode import UNICODE_EMOJI
from emot.emo_unicode import EMOTICONS_EMO

In [14]:
def converting_emojis(text):
  for emot in UNICODE_EMOJI:
    text=text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
  return text
converting_emojis(text1)

'What are you saying face_with_tears_of_joy, I am the boss smiling_face_with_sunglasses, and why are you so pensive_face'

In [15]:
def emoji_removal(string):
    emoji_unicodes = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
    "]+", flags=re.UNICODE)

    return emoji_unicodes.sub(r'', string)
emoji_removal(text1)

'What are you saying , I am the boss , and why are you so '

In [16]:
sent = ["What are you saying 😂", "I am the boss 😎", "why are you so 😔"]
import pandas as pd
df = pd.DataFrame({'text': sent})
df.head()


,text
0,What are you saying 😂
1,I am the boss 😎
2,why are you so 😔
